In [1]:
import numpy as np
import pandas as pd
import time

df = pd.read_csv('LE.csv')
df = df [['Country','Year','Status','Life expectancy']]

In [2]:
df.head()

,Country,Year,Status,Life expectancy
0,Afghanistan,2015,Developing,65.0
1,Afghanistan,2014,Developing,59.9
2,Afghanistan,2013,Developing,59.9
3,Afghanistan,2012,Developing,59.5
4,Afghanistan,2011,Developing,59.2


In [8]:
from catboost import CatBoostRegressor, Pool

In [9]:
X = df.drop(columns='Life expectancy')
y = df['Life expectancy']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)

In [3]:
pool_train = Pool(X_train, y_train,
                  cat_features = ['Country','Year','Status'])

pool_test = Pool(X_test, cat_features = ['Country','Year','Status'])


#CatBoost

import time
start = time.time()

cbr = CatBoostRegressor(iterations=100,max_depth=2)

cbr.fit(pool_train)
y_pred = cbr.predict(X_test)

from sklearn.metrics import r2_score as RSquared


cb_rsquared = np.sqrt(RSquared(y_test, y_pred))
print("R Squared for CatBoost: ", np.mean(cb_rsquared))

end = time.time()
diff = end - start
print('Execution time:', diff)


Learning rate set to 0.304491
0:	learn: 7.9652996	total: 61.2ms	remaining: 6.06s
1:	learn: 6.8750145	total: 61.6ms	remaining: 3.02s
2:	learn: 6.3649201	total: 62ms	remaining: 2.01s
3:	learn: 6.0458584	total: 62.4ms	remaining: 1.5s
4:	learn: 5.7968748	total: 63.5ms	remaining: 1.21s
5:	learn: 5.6849516	total: 63.9ms	remaining: 1s
6:	learn: 5.5930909	total: 64.2ms	remaining: 853ms
7:	learn: 5.5501911	total: 64.5ms	remaining: 742ms
8:	learn: 5.5157724	total: 65.1ms	remaining: 658ms
9:	learn: 5.4922729	total: 65.7ms	remaining: 591ms
10:	learn: 5.4620773	total: 66ms	remaining: 534ms
11:	learn: 5.4609070	total: 66.4ms	remaining: 487ms
12:	learn: 5.4574215	total: 66.7ms	remaining: 446ms
13:	learn: 5.4314251	total: 67ms	remaining: 412ms
14:	learn: 5.4191324	total: 67.3ms	remaining: 382ms
15:	learn: 5.4029769	total: 67.6ms	remaining: 355ms
16:	learn: 5.3918803	total: 67.9ms	remaining: 332ms
17:	learn: 5.3861049	total: 68.3ms	remaining: 311ms
18:	learn: 5.3766558	total: 68.7ms	remaining: 293ms
19

/opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.9/site-packages/catboost/core.py:1325: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, pairs, weight,


In [4]:
#XGBoost
import xgboost as xgb
from sklearn import preprocessing
X = df.drop(columns='Life expectancy')
y = df['Life expectancy']

lbl = preprocessing.LabelEncoder()
#Country','Year','Status
X['Country'] = lbl.fit_transform(X['Country'].astype(str))
X['Year'] = lbl.fit_transform(X['Year'].astype(str))
X['Status'] = lbl.fit_transform(X['Status'].astype(str))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)

start = time.time()
#X_train["Species"].astype("category")
xgbr = xgb.XGBRegressor()

xgbr.fit(X_train, y_train)
y_pred = xgbr.predict(X_test)

xgb_rmse = np.sqrt(RSquared(y_test, y_pred))
print("R Squared for XGBoost: ", np.mean(xgb_rmse))

end = time.time()
diff = end - start
print('Execution time:', diff)

R Squared for XGBoost:  0.9787312801955712
Execution time: 0.10053706169128418


In [5]:
import lightgbm
X = df.drop(columns='Life expectancy')
y = df['Life expectancy']
obj_feat = list(X.loc[:, X.dtypes == 'object'].columns.values)

for feature in obj_feat:
    X[feature] = pd.Series(X[feature], dtype="category")

start = time.time()

lgbmr = lightgbm.LGBMRegressor()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)

lgbmr.fit(X_train, y_train)
y_pred = lgbmr.predict(X_test)

lgbm_rsquared = np.sqrt(RSquared(y_test, y_pred))
print()
print("R squared for LightGBM: ", np.mean(lgbm_rsquared))

end = time.time()
diff = end - start
print('Execution time:', diff)


R squared for LightGBM:  0.9767455250796998
Execution time: 0.0787811279296875
